In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *

In [3]:
from nlptools.basics import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [4]:
from twinews.yfnotebooks.dssm.graph import Graph
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from twinews.yfnotebooks.dssm.load_data import hashIndex,load_hashed_data

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [6]:
logger = Logger(tmpDir('logs') + "/dssm_yf.log") if isNotebook else Logger("dssm_yf-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [7]:
modelName = "DSSM_yf"

In [8]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL’"] = "3"

# config

In [9]:
config = \
{
    'splitVersion': 2,
    
    # for input data
    'maxUsers': 10 if TEST else None, # Sub-sampling
    'maxDocuments': 100 if TEST else 10000,
    'useExtraNews': 0 if TEST else None, # None = unlimited, 0 = no extra news
    'minDF': 1 / 500 if TEST else 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 300, # Remove top 300 voc elements
    'lowercase': False if TEST else True,
    'doLemmatization': False if TEST else False,
    #'useTFIDF': True,
    
   # for model    
    'seq_length' : 7427,
    'char_embedding_size': 100,
    'learning_rate': 0.0005,
    'keep_prob': 0.7,
    'vocab_size': 7901,
    'class_size': 2,
    'epochs': 10,
    'batch_size': 100,
    
    # for ouput
    'similarity': 'cosine',

    'historyRef': 0.3, # 1, 1.0, 0.5, 0.3, 3, 10
}

# get data

In [11]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0,
                       maxUsers=config['maxUsers'])
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 29.44s | message: Eval data loaded


{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


--> toc total duration: 29.66s | message: Got Twinews evaluation data


In [12]:
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [13]:
# train news as query
newsList = trainNewsList + testNewsList
log(str(len(newsList)) + " urls for newsList:\n" + b(newsList), logger=logger)

9690 urls for newsList:
[
  https://www.texastribune.org/2017/11/13/texas-capitol-victims-sexual-harassment-must-fend-themselves,
  http://politi.co/2zAjExo,
  ...,
  https://www.theguardian.com/artanddesign/2018/jan/03/marilyn-stafford-best-photograph-albert-einstei,
  http://chicago.suntimes.com/sports/bears-coaching-candidate-interview-tracker/amp
]


In [14]:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger=logger)

twinews news (version 1.0) initialised.
  0% [                    ]
 10% [==                  ] (5m 5.639s left)
 20% [====                ] (4m 29.159s left)
 30% [======              ] (3m 53.94s left)
 40% [========            ] (3m 19.589s left)
 50% [==========          ] (2m 46.419s left)
 60% [============        ] (2m 12.853s left)
 70% [==============      ] (1m 39.505s left)
 80% [================    ] (1m 6.439s left)
 90% [==================  ] (33.19s left)
100% [====================] (total duration: 5m 31.47s, mean duration: 0.034s)
[
  [ [ Disgust, overwhelmed, ..., hand, . ], [ A, Capitol, ..., her, . ], ..., [ Correction, :, ..., outdated, . ], [ Only, the, ..., exists, . ] ],
  [ [ WEST, CHESTER, ..., southpaw, . ], [ Maybe, it, ..., it, . ], ..., [ ", Be, ..., says, . ], [ Tim, Alberta, ..., Magazine, . ] ],
  ...,
  [ [ ', It, ..., in, __int_4__ ], [ Photograph, :, ..., acting, . ], ..., [ This, was, ..., took, . ], [ So, it, ..., photographer, . ] ],
  [ [ The, Be

In [15]:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ Disgust, overwhelmed, ..., exists, . ], [ WEST, CHESTER, ..., Magazine, . ], ..., [ ', It, ..., photographer, . ], [ The, Bears, ..., weekend, . ] ]


In [16]:
if TEST:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

Lower casing   0% [                    ]
Lower casing  10% [==                  ] (2.16s left)
Lower casing  20% [====                ] (1.72s left)
Lower casing  30% [======              ] (1.47s left)
Lower casing  40% [========            ] (1.23s left)
Lower casing  50% [==========          ] (1.04s left)
Lower casing  60% [============        ] (0.82s left)
Lower casing  70% [==============      ] (0.617s left)
Lower casing  80% [================    ] (0.412s left)
Lower casing  90% [==================  ] (0.205s left)
Lower casing 100% [====================] (total duration: 2.05s, mean duration: 0s)
[ [ disgust, overwhelmed, ..., exists, . ], [ west, chester, ..., magazine, . ], ..., [ ', it, ..., photographer, . ], [ the, bears, ..., weekend, . ] ]


In [17]:
if TEST:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization  10% [==                  ] (33.75s left)
Lemmatization  20% [====                ] (23.52s left)
Lemmatization  30% [======              ] (18.666s left)
Lemmatization  40% [========            ] (15.225s left)
Lemmatization  50% [==========          ] (12.43s left)
Lemmatization  60% [============        ] (9.653s left)
Lemmatization  70% [==============      ] (7.148s left)
Lemmatization  80% [================    ] (4.719s left)
Lemmatization  90% [==================  ] (2.325s left)
Lemmatization 100% [====================] (total duration: 23.13s, mean duration: 0.002s)
[ [ disgust, overwhelmed, ..., exists, . ], [ west, chester, ..., magazine, . ], ..., [ ', it, ..., photographer, . ], [ the, bear, ..., weekend, . ] ]


In [18]:
docs = filterCorpus(docs, minDF=1/2000, maxDF=300,
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc removed because of minDF (66696 elements):
{ 00s, 0640l, 0cean, 0day, 0f, 0x, 1, 10, 100, 1000th, ..., zzz, zzzs, 💗, 😂, 😃, 😉, 😍, 😒, 😢, 🙌 }
Voc removed because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., without, woman, work, working, world, would, year, yet, you, your }
70.36% of voc will be removed.
[ [ disgust, overwhelmed, ..., longer, exists ], [ west, chester, ..., politico, magazine ], ..., [ ever, photograph, ..., became, photographer ], [ bear, beginning, ..., round, weekend ] ]


In [19]:
def toStr(sentencesList):
    for i in range(len(sentencesList)):
        sentencesList[i] = ' '.join(sentencesList[i])
    return sentencesList

corpus = toStr(docs)
for corp in corpus: assert len(corp) > 0
bp(corpus,logger)

[
  disgust overwhelmed felt tongue hand capitol staffer large party celebrating legislative session ago,
  west chester ohio swing golf club hander maybe metaphor conservative politician turned liberal crunc,
  ...,
  ever photograph albert einstein jersey photograph marilyn stafford living york hoping break acting c,
  bear beginning coaching search replace john fox follow along bear interviewed defensive coordinator 
]


In [20]:
urlDocs = dict()
for i in range(len(newsList)):
    urlDocs[newsList[i]] = corpus[i]
bp(urlDocs, 2, logger)

{
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: estimated __float_3__ united __float_1__ ago economy whole grew healthy pace workforce growth unifor,
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: looking resolution extra pound holiday surprisingly looking invest nascent field wearable tech fitne,
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: wal mart nyse decided raise minimum wage pay worker hour following footstep target nyse september de,
  http://a.msn.com/00/en-us/BBHoy0Q?ocid=st: richest earth became trillion richer gain stock market shrugged economic social political division r,
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: brought stunning rare wolf moon light sky monday evening lunar cycle cycle align astronomical treat ,
  ...,
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: reason behind rebound economy explained word charge confirmati

In [21]:
urlHasedDocs = dict()
for key in urlDocs.keys():
    temp = []
    temp.append(urlDocs[key])
    urlHasedDocs[key], _  = hashIndex(temp, ' ')
bp(urlHasedDocs)

{
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: (0, 54) 1.0
(0, 89) 1.0
(0, 93) 2.0
(0, 98) 1.0
(0, 117) 1.0
(0, 124) 2.0
(0, 128) 1.0
(0, 131) 1.0
...,
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: (0, 1) 1.0
(0, 23) 1.0
(0, 32) 5.0
(0, 47) 1.0
(0, 52) 1.0
(0, 54) 1.0
(0, 73) 9.0
(0, 83) 1.0
(0, 8...,
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: (0, 14) 2.0
(0, 61) 14.0
(0, 62) 1.0
(0, 63) 1.0
(0, 78) 1.0
(0, 93) 1.0
(0, 124) 1.0
(0, 128) 1.0
(...,
  http://a.msn.com/00/en-us/BBHoy0Q?ocid=st: (0, 22) 2.0
(0, 23) 1.0
(0, 24) 1.0
(0, 32) 12.0
(0, 54) 2.0
(0, 61) 2.0
(0, 62) 2.0
(0, 63) 2.0
(0,...,
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: (0, 1) 1.0
(0, 29) 1.0
(0, 32) 1.0
(0, 52) 1.0
(0, 59) 3.0
(0, 61) 1.0
(0, 67) 1.0
(0, 78) 1.0
(0, 8...,
  ...,
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: (0, 32) 2.0
(0, 38) 1.0
(0, 54) 1.0
(0, 59) 1.0

# news classification

In [22]:
# for userId in trainUsers.keys():
query = np.zeros((1,7427))
for url in evalData['trainUsers']['1198644368']:
    query += urlHasedDocs[url].toarray() # sum the history urls of users
bp(query,2)

[[ 0. 25.  0. ...  0. 21.  0.]]


In [23]:
# test
# p, h, y = load_hashed_data('/home/yuting/PycharmProjects/data/dssm_test_dev_sub.csv', data_size=None)
p = np.tile(query,(1000,1))  # duplicate for 1000 times to match the number of candidates

# get h
h = np.zeros((1,7427))
for url in evalData['candidates']['1198644368'][0]:
    h_temp = urlHasedDocs[url].toarray()
    h = np.vstack((h,h_temp))
h = h[1:]               # cut off the first element
    
# h = urlHasedDocs['http://a.msn.com/01/en-ie/BBHzIAS?ocid=st'].toarray()
y = np.zeros((1000))

In [24]:
def computeSimilarity(p,h,y):
    
    tf.reset_default_graph() 
    model = Graph()
    saver = tf.train.Saver()

    with tf.Session()as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt')
        loss, acc, logit = sess.run([model.loss, model.acc, model.logits],
                             feed_dict={model.p: p,
                                        model.h: h,
                                        model.y: y,
                                        model.keep_prob: 1})

        #print('loss: ', loss, ' acc:', acc)
        return logit
logit = computeSimilarity(p,h,y)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt


In [73]:
# rankings of the candidates
rankings = dict()
for usr in trainUsers.keys():
    # get historical data as query(p)
    query = np.zeros((1,7427))
    for url in evalData['trainUsers'][usr]:
        query += urlHasedDocs[url].toarray()
        p = np.tile(query,(1000,1)) 
    usrRankings = []
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h_doc as a list(h)
        h = np.zeros((1,7427))
        for url in candidates:
            h_temp = urlHasedDocs[url].toarray()
            h = np.vstack((h,h_temp))
        h = h[1:]                                   # cut off the first element
        y = np.zeros((1000))
        scoresCandidates = []
        logit = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,list(logit[:,0]))
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

In [74]:
bp(rankings,2)

{
  1198644368: 
  [
    [
      http://tinyurl.com/ycvt6rp5,
      https://www.bizjournals.com/nashville/news/2018/01/11/who-would-play-nashvilles-ceos-in-the-movies-c,
      ...,
      https://www.washingtonpost.com/lifestyle/magazine/dave-barrys-year-in-review-russia-mania-covfefe-an,
      https://www.hrw.org/report/2007/09/11/no-easy-answers/sex-offender-laws-us
    ]
  ],
  21379050: 
  [
    [
      https://www.wsj.com/articles/the-great-rules-rollback-1514237372,
      http://www.telegraph.co.uk/news/2017/12/31/ministers-use-artificial-intelligence-target-mass-benefit,
      ...,
      http://time.com/69316/basic/,
      http://ow.ly/YTHL30hlDxp
    ]
  ],
  2617447752: 
  [
    [
      https://www.csindy.com/SideDish/archives/2017/12/29/mccabes-to-close-become-streetcar-520,
      https://pamboyd.wordpress.com/2018/01/02/while-thinking-over-2017/,
      ...,
      https://www.theatlantic.com/magazine/archive/1901/01/the-reconstruction-of-the-southern-states/52003,
      http:/

In [77]:
checkRankings(rankings,evalData['candidates'],maxUsers=None)

In [76]:
addRanking(modelName, rankings, config, logger=logger)

Exception type: <class 'Exception'>
Exception: DSSM_yf-d9604 is already in the database.
Traceback (most recent call last):
  File "/home/yuting/PycharmProjects/Twinews/twinews/utils.py", line 341, in addRanking
    raise Exception(key + " is already in the database.",)
Exception: DSSM_yf-d9604 is already in the database.



In [56]:
modelName

'DSSM_yf'

# Extra notes

In [36]:
a = np.array([[1,2,3,4,5]])
a.shape

(1, 5)

In [51]:
b = np.tile(a,(5,1))

In [59]:
b.shape

(6, 5)

In [ ]:
# rankings of the candidates
rankings = dict()
for usr in trainUsers.keys():
    query = np.zeros((1,7427))
    for url in evalData['trainUsers'][usr]:
        query += urlHasedDocs[url].toarray()
    usrRankings = []
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        scoresCandidates = []
        for i in range(len(candidates)):
            candidateHashed = urlHasedDocs[candidates[i]].toarray()
            logit = computeSimilarity(query, candidateHashed, 0)
            score = logit[0][0]
            # score = 1
            scoresCandidates.append((candidates[i], score))
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings